                 Name: Shreya            Andrew ID: sshreya             Zillow Zestimate Prediction                             

### Zillow, a real-estate listing service that includes information for home buyers and sellers, the listing history for properties and a Zestimate, which is a prediction of the current home value of a property. The Zestimate is based on numerous factors, including the location, size, number of bedrooms and bathrooms, and so on, for a given home. 

## Objective : Predict the log loss of the Zestimate and target home prices to improve on the Zestimate.

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import pickle
import warnings
warnings.filterwarnings("ignore")

## Data preprocessing
### Part 1: processing and merging the data

In [4]:
train_df_17 = pd.read_csv("train_2017.csv")
train_df_17

,parcelid,logerror,transactiondate
0,14297519,0.025595,2017-01-01
1,17052889,0.055619,2017-01-01
2,14186244,0.005383,2017-01-01
3,12177905,-0.103410,2017-01-01
4,10887214,0.006940,2017-01-01
...,...,...,...
77608,10833991,-0.002245,2017-09-20
77609,11000655,0.020615,2017-09-20
77610,17239384,0.013209,2017-09-21
77611,12773139,0.037129,2017-09-21


In [5]:
train_df_16 = pd.read_csv("train_2016.csv")
train_df_16

,parcelid,logerror,transactiondate
0,11016594,0.0276,2016-01-01
1,14366692,-0.1684,2016-01-01
2,12098116,-0.0040,2016-01-01
3,12643413,0.0218,2016-01-02
4,14432541,-0.0050,2016-01-02
...,...,...,...
90270,10774160,-0.0356,2016-12-30
90271,12046695,0.0070,2016-12-30
90272,12995401,-0.2679,2016-12-30
90273,11402105,0.0602,2016-12-30


In [6]:
properties_df_16 = pd.read_csv("properties_2016.csv")
properties_df_16

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.17,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2985212,168176230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985213,14273630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985214,168040630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985215,168040830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
properties_df_17 = pd.read_csv("properties_2017.csv")
properties_df_17

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2016.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,5.0,NaN,NaN,NaN,...,1.0,NaN,660680.0,1434941.0,2016.0,774261.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,6.0,NaN,NaN,...,1.0,NaN,580059.0,1174475.0,2016.0,594416.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,1.0,NaN,196751.0,440101.0,2016.0,243350.0,5725.17,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2985212,12170430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985213,11422430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985214,17203030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985215,14242830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
print(len(train_df_17))
print(len(train_df_16))
print(len(train_df_17['parcelid'].unique()))
print(len(train_df_16['parcelid'].unique()))
print(len(properties_df_16['parcelid'].unique()))
print(len(properties_df_17['parcelid'].unique()))

77613
90275
77414
90150
2985217
2985217


#### Merging the properties details with the train data from 2016 and 2017, removing the duplicates

In [9]:
duplicateTrainDF_16 = train_df_16[train_df_16.duplicated(["parcelid"],keep=False)]
duplicateTrainDF_16["parcelid"].value_counts()

11842707    3
13850164    2
11105038    2
11602482    2
12613442    2
           ..
11633771    2
17151530    2
10911172    2
14678446    2
14367791    2
Name: parcelid, Length: 124, dtype: int64

In [10]:
duplicateTrainDF_16.loc[duplicateTrainDF_16["parcelid"]==11842707]

,parcelid,logerror,transactiondate
55794,11842707,-0.0284,2016-07-14
55795,11842707,0.0573,2016-08-22
55796,11842707,0.2078,2016-09-29


In [11]:
# getting the latest transaction of the property
unique_train_data_16 = train_df_16.sort_values("transactiondate").drop_duplicates("parcelid",keep = "last")
unique_train_data_16

,parcelid,logerror,transactiondate
0,11016594,0.0276,2016-01-01
1,14366692,-0.1684,2016-01-01
2,12098116,-0.0040,2016-01-01
3,12643413,0.0218,2016-01-02
4,14432541,-0.0050,2016-01-02
...,...,...,...
90207,12819764,0.1151,2016-12-30
90206,11206485,0.0227,2016-12-30
90205,11831984,-0.0576,2016-12-30
90214,14118061,-0.0020,2016-12-30


In [12]:
duplicateTrainDF_17 = train_df_17[train_df_17.duplicated(["parcelid"],keep=False)]
duplicateTrainDF_17["parcelid"].value_counts()

11991059    3
10857130    3
12612211    3
11393337    2
12892594    2
           ..
14079874    2
11401519    2
12607366    2
12532988    2
10984080    2
Name: parcelid, Length: 196, dtype: int64

In [13]:
duplicateTrainDF_17.loc[duplicateTrainDF_17["parcelid"]==11991059]

,parcelid,logerror,transactiondate
43840,11991059,2.619876,2017-06-06
43841,11991059,2.670239,2017-06-09
43842,11991059,2.508444,2017-06-13


In [14]:
# getting the latest transaction of the property
unique_train_data_17 = train_df_17.sort_values("transactiondate").drop_duplicates("parcelid",keep = "last")
unique_train_data_17

,parcelid,logerror,transactiondate
0,14297519,0.025595,2017-01-01
1,17052889,0.055619,2017-01-01
2,14186244,0.005383,2017-01-01
3,12177905,-0.103410,2017-01-01
4,10887214,0.006940,2017-01-01
...,...,...,...
77608,10833991,-0.002245,2017-09-20
77609,11000655,0.020615,2017-09-20
77611,12773139,0.037129,2017-09-21
77610,17239384,0.013209,2017-09-21


In [15]:
unique_train_16_ls= list(unique_train_data_16['parcelid'])
print(len(unique_train_16_ls))
#getting only those properties for which we have train data
train_prop_16 = properties_df_16[properties_df_16['parcelid'].isin(unique_train_16_ls)]
train_prop_16

90150


,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
362,17073783,NaN,NaN,NaN,2.5,3.0,NaN,NaN,2.5,NaN,...,2.0,NaN,115087.0,191811.0,2015.0,76724.0,2015.06,NaN,NaN,6.111002e+13
428,17088994,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,NaN,...,1.0,NaN,143809.0,239679.0,2015.0,95870.0,2581.30,NaN,NaN,6.111002e+13
470,17100444,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,NaN,...,1.0,NaN,33619.0,47853.0,2015.0,14234.0,591.64,NaN,NaN,6.111001e+13
480,17102429,NaN,NaN,NaN,1.5,2.0,NaN,NaN,1.5,NaN,...,1.0,NaN,45609.0,62914.0,2015.0,17305.0,682.78,NaN,NaN,6.111001e+13
507,17109604,NaN,NaN,NaN,2.5,4.0,NaN,NaN,2.5,NaN,...,2.0,NaN,277000.0,554000.0,2015.0,277000.0,5886.92,NaN,NaN,6.111001e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2973529,11876798,1.0,NaN,NaN,3.0,4.0,NaN,4.0,3.0,NaN,...,NaN,NaN,388582.0,596082.0,2015.0,207500.0,7335.81,NaN,NaN,6.037201e+13
2973663,12808516,NaN,NaN,NaN,2.0,3.0,NaN,7.0,2.0,NaN,...,NaN,NaN,86209.0,178408.0,2015.0,92199.0,2441.74,NaN,NaN,6.037407e+13
2973689,12010248,1.0,NaN,NaN,2.0,2.0,NaN,4.0,2.0,NaN,...,NaN,NaN,129000.0,420000.0,2015.0,291000.0,5070.41,NaN,NaN,6.037191e+13
2973726,14310905,NaN,NaN,NaN,2.5,3.0,NaN,NaN,2.5,NaN,...,NaN,NaN,389474.0,1215816.0,2015.0,826342.0,12508.30,NaN,NaN,NaN


In [16]:
unique_train_17_ls= list(unique_train_data_17['parcelid'])
print(len(unique_train_17_ls))
#getting only those properties for which we have train data
train_prop_17 = properties_df_17[properties_df_17['parcelid'].isin(unique_train_17_ls)]
train_prop_17

77414


,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
349,17054981,NaN,NaN,NaN,5.0,4.0,NaN,NaN,5.0,NaN,...,2.0,NaN,501928.0,872850.0,2016.0,370922.0,9673.46,NaN,NaN,6.111001e+13
355,17055743,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,66.0,...,1.0,NaN,130845.0,436157.0,2016.0,305312.0,5538.80,NaN,NaN,6.111001e+13
383,17068109,NaN,NaN,NaN,1.5,3.0,NaN,NaN,1.5,66.0,...,1.0,NaN,193413.0,286606.0,2016.0,93193.0,2987.36,NaN,NaN,6.111001e+13
406,17073952,NaN,NaN,NaN,2.0,2.0,NaN,NaN,2.0,NaN,...,1.0,NaN,90357.0,258888.0,2016.0,168531.0,2706.24,NaN,NaN,6.111002e+13
423,17078502,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,NaN,...,1.0,NaN,148752.0,592930.0,2016.0,444178.0,6220.70,NaN,NaN,6.111002e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2985043,11928618,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985044,10813418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985111,11934023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985145,11490025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Merge the train data with property data and remove the duplicates

In [17]:
#merge train with properties data
all_16 = pd.merge(train_prop_16, unique_train_data_16, on='parcelid')
all_16

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate
0,17073783,NaN,NaN,NaN,2.5,3.0,NaN,NaN,2.5,NaN,...,115087.0,191811.0,2015.0,76724.0,2015.06,NaN,NaN,6.111002e+13,0.0953,2016-01-27
1,17088994,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,NaN,...,143809.0,239679.0,2015.0,95870.0,2581.30,NaN,NaN,6.111002e+13,0.0198,2016-03-30
2,17100444,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,NaN,...,33619.0,47853.0,2015.0,14234.0,591.64,NaN,NaN,6.111001e+13,0.0060,2016-05-27
3,17102429,NaN,NaN,NaN,1.5,2.0,NaN,NaN,1.5,NaN,...,45609.0,62914.0,2015.0,17305.0,682.78,NaN,NaN,6.111001e+13,-0.0566,2016-06-07
4,17109604,NaN,NaN,NaN,2.5,4.0,NaN,NaN,2.5,NaN,...,277000.0,554000.0,2015.0,277000.0,5886.92,NaN,NaN,6.111001e+13,0.0573,2016-08-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90145,11876798,1.0,NaN,NaN,3.0,4.0,NaN,4.0,3.0,NaN,...,388582.0,596082.0,2015.0,207500.0,7335.81,NaN,NaN,6.037201e+13,0.0276,2016-04-15
90146,12808516,NaN,NaN,NaN,2.0,3.0,NaN,7.0,2.0,NaN,...,86209.0,178408.0,2015.0,92199.0,2441.74,NaN,NaN,6.037407e+13,-0.1188,2016-06-07
90147,12010248,1.0,NaN,NaN,2.0,2.0,NaN,4.0,2.0,NaN,...,129000.0,420000.0,2015.0,291000.0,5070.41,NaN,NaN,6.037191e+13,0.0843,2016-06-30
90148,14310905,NaN,NaN,NaN,2.5,3.0,NaN,NaN,2.5,NaN,...,389474.0,1215816.0,2015.0,826342.0,12508.30,NaN,NaN,NaN,-0.0050,2016-03-31


In [18]:
#merge train with properties data
all_17 = pd.merge(train_prop_17, unique_train_data_17, on='parcelid')
all_17

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate
0,17054981,NaN,NaN,NaN,5.0,4.0,NaN,NaN,5.0,NaN,...,501928.0,872850.0,2016.0,370922.0,9673.46,NaN,NaN,6.111001e+13,-0.013099,2017-06-15
1,17055743,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,66.0,...,130845.0,436157.0,2016.0,305312.0,5538.80,NaN,NaN,6.111001e+13,0.073985,2017-07-26
2,17068109,NaN,NaN,NaN,1.5,3.0,NaN,NaN,1.5,66.0,...,193413.0,286606.0,2016.0,93193.0,2987.36,NaN,NaN,6.111001e+13,0.071886,2017-07-28
3,17073952,NaN,NaN,NaN,2.0,2.0,NaN,NaN,2.0,NaN,...,90357.0,258888.0,2016.0,168531.0,2706.24,NaN,NaN,6.111002e+13,0.305680,2017-06-02
4,17078502,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,NaN,...,148752.0,592930.0,2016.0,444178.0,6220.70,NaN,NaN,6.111002e+13,-0.073787,2017-07-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77409,11928618,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.026665,2017-05-12
77410,10813418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.008083,2017-01-03
77411,11934023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.016600,2017-03-23
77412,11490025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.108073,2017-03-23


In [19]:
#merge 2016 and 2017 train data
all = pd.concat([all_16,all_17])
all

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate
0,17073783,NaN,NaN,NaN,2.5,3.0,NaN,NaN,2.5,NaN,...,115087.0,191811.0,2015.0,76724.0,2015.06,NaN,NaN,6.111002e+13,0.095300,2016-01-27
1,17088994,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,NaN,...,143809.0,239679.0,2015.0,95870.0,2581.30,NaN,NaN,6.111002e+13,0.019800,2016-03-30
2,17100444,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,NaN,...,33619.0,47853.0,2015.0,14234.0,591.64,NaN,NaN,6.111001e+13,0.006000,2016-05-27
3,17102429,NaN,NaN,NaN,1.5,2.0,NaN,NaN,1.5,NaN,...,45609.0,62914.0,2015.0,17305.0,682.78,NaN,NaN,6.111001e+13,-0.056600,2016-06-07
4,17109604,NaN,NaN,NaN,2.5,4.0,NaN,NaN,2.5,NaN,...,277000.0,554000.0,2015.0,277000.0,5886.92,NaN,NaN,6.111001e+13,0.057300,2016-08-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77409,11928618,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.026665,2017-05-12
77410,10813418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.008083,2017-01-03
77411,11934023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.016600,2017-03-23
77412,11490025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.108073,2017-03-23


In [20]:
all['parcelid'].nunique()

165210

In [21]:
#remove duplicate values after merging based on assessmentyear
duplicateDF = all[all.duplicated(["parcelid"],keep=False)]
print(len(duplicateDF))
print(len(duplicateDF['parcelid'].unique()))
duplicateDF["parcelid"].value_counts()

4708
2354


17088994    2
11349196    2
10777629    2
10786030    2
10960489    2
           ..
11835073    2
12220614    2
12740626    2
12895699    2
14693352    2
Name: parcelid, Length: 2354, dtype: int64

In [22]:
duplicateDF.loc[duplicateDF["parcelid"]==11349196]

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate
59691,11349196,1.0,NaN,NaN,3.0,3.0,NaN,4.0,3.0,NaN,...,233000.0,387000.0,2015.0,154000.0,9735.03,NaN,NaN,6.037920e+13,0.025700,2016-01-25
51458,11349196,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,NaN,...,233000.0,387000.0,2016.0,154000.0,9735.03,NaN,NaN,6.037920e+13,0.031158,2017-01-06


In [23]:
#latest
all = all.sort_values("assessmentyear").drop_duplicates("parcelid",keep = "last")
all.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate
0,17073783,NaN,NaN,NaN,2.5,3.0,NaN,NaN,2.5,NaN,...,115087.0,191811.0,2015.0,76724.0,2015.06,NaN,NaN,6.111002e+13,0.0953,2016-01-27
60104,14762478,NaN,NaN,NaN,3.0,4.0,NaN,NaN,3.0,NaN,...,241544.0,397303.0,2015.0,155759.0,4099.00,NaN,NaN,6.059063e+13,0.8519,2016-01-06
60103,14743592,NaN,NaN,NaN,2.0,2.0,NaN,NaN,2.0,NaN,...,115279.0,211050.0,2015.0,95771.0,2863.96,NaN,NaN,6.059053e+13,-0.1393,2016-09-26
60102,14729860,NaN,NaN,NaN,2.0,2.0,NaN,NaN,2.0,NaN,...,94935.0,298480.0,2015.0,203545.0,3081.24,NaN,NaN,6.059042e+13,-0.0212,2016-03-16
60101,14721734,NaN,NaN,NaN,2.0,2.0,NaN,NaN,2.0,NaN,...,111769.0,354427.0,2015.0,242658.0,4094.74,NaN,NaN,6.059076e+13,0.0459,2016-06-16


In [24]:
all.describe()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,logerror
count,1.652100e+05,53057.000000,459.000000,92.000000,165177.000000,165177.000000,30.000000,105366.000000,163422.000000,1262.0,...,164182.000000,37610.000000,1.646920e+05,1.651750e+05,165177.000000,1.651740e+05,165166.000000,4529.000000,1.643410e+05,165210.000000
mean,1.299793e+07,1.814897,7.305011,697.336957,2.291121,3.043608,3.966667,6.017444,2.315218,66.0,...,1968.668003,1.438421,1.848479e+05,4.738407e+05,2015.468473,2.895351e+05,5999.531543,13.817841,6.049424e+13,0.012928
std,3.010371e+06,2.971235,2.745897,586.844051,1.000715,1.148108,0.182574,1.884160,0.977816,0.0,...,23.748635,0.544636,2.192159e+05,6.033909e+05,0.499007,4.465328e+05,7214.795709,2.085555,1.062366e+12,0.164396
min,1.071174e+07,1.000000,2.000000,38.000000,0.000000,0.000000,3.000000,1.000000,1.000000,66.0,...,1824.000000,1.000000,4.400000e+01,2.200000e+01,2015.000000,2.200000e+01,19.920000,3.000000,6.037101e+13,-4.655420
25%,1.154960e+07,1.000000,7.000000,278.250000,2.000000,2.000000,4.000000,4.000000,2.000000,66.0,...,1953.000000,1.000000,8.295700e+04,2.036995e+05,2015.000000,8.449100e+04,2809.977500,14.000000,6.037312e+13,-0.025300
50%,1.254324e+07,1.000000,7.000000,594.000000,2.000000,3.000000,4.000000,7.000000,2.000000,66.0,...,1970.000000,1.000000,1.345760e+05,3.509280e+05,2015.000000,1.984640e+05,4512.360000,14.000000,6.037604e+13,0.006000
75%,1.422105e+07,1.000000,7.000000,822.000000,3.000000,4.000000,4.000000,7.000000,3.000000,66.0,...,1987.000000,2.000000,2.148290e+05,5.550110e+05,2016.000000,3.560658e+05,6927.247500,15.000000,6.059042e+13,0.038300
max,1.676893e+08,13.000000,21.000000,3560.000000,20.000000,16.000000,4.000000,12.000000,20.000000,66.0,...,2016.000000,6.000000,1.142179e+07,4.906124e+07,2016.000000,4.895220e+07,586639.300000,99.000000,4.830301e+14,5.262999


In [25]:
# checking datatypes of each column
all.dtypes

parcelid                          int64
airconditioningtypeid           float64
architecturalstyletypeid        float64
basementsqft                    float64
bathroomcnt                     float64
bedroomcnt                      float64
buildingclasstypeid             float64
buildingqualitytypeid           float64
calculatedbathnbr               float64
decktypeid                      float64
finishedfloor1squarefeet        float64
calculatedfinishedsquarefeet    float64
finishedsquarefeet12            float64
finishedsquarefeet13            float64
finishedsquarefeet15            float64
finishedsquarefeet50            float64
finishedsquarefeet6             float64
fips                            float64
fireplacecnt                    float64
fullbathcnt                     float64
garagecarcnt                    float64
garagetotalsqft                 float64
hashottuborspa                   object
heatingorsystemtypeid           float64
latitude                        float64


## Data preprocessing
### Part 2: Analyze the data and make appropriate preprocessing decisions (imputation, converting to categorical, etc.).

In [26]:
# checking is there are any records if label is null or not
print(all['logerror'].isnull().sum())

0


In [27]:
all.columns

Index(['parcelid', 'airconditioningtypeid', 'architecturalstyletypeid',
       'basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingclasstypeid',
       'buildingqualitytypeid', 'calculatedbathnbr', 'decktypeid',
       'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet',
       'finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15',
       'finishedsquarefeet50', 'finishedsquarefeet6', 'fips', 'fireplacecnt',
       'fullbathcnt', 'garagecarcnt', 'garagetotalsqft', 'hashottuborspa',
       'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet',
       'poolcnt', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7',
       'propertycountylandusecode', 'propertylandusetypeid',
       'propertyzoningdesc', 'rawcensustractandblock', 'regionidcity',
       'regionidcounty', 'regionidneighborhood', 'regionidzip', 'roomcnt',
       'storytypeid', 'threequarterbathnbr', 'typeconstructiontypeid',
       'unitcnt', 'yardbuildingsqft17', 'yardbuildin

In [28]:
# check correlation between the columns
corr = all.corr()
corr.style.background_gradient(cmap='coolwarm')

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertylandusetypeid,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,logerror
parcelid,1.000000,0.218169,-0.004165,0.332278,-0.005498,0.005834,-0.115675,-0.005154,0.006049,nan,-0.006122,0.021634,0.031831,0.466207,-0.027557,-0.006513,0.084203,0.545710,0.015418,-0.042649,0.016841,-0.025188,0.159608,-0.196284,-0.016275,-0.078166,nan,-0.003703,nan,nan,nan,0.037193,0.545240,-0.018634,-0.423128,0.340547,0.045932,0.399080,nan,-0.024849,0.030340,0.020517,-0.007611,0.148989,0.151323,0.035834,0.020927,0.008841,0.002811,0.002775,-0.012058,-0.004151,0.138293,0.012268
airconditioningtypeid,0.218169,1.000000,-0.072694,nan,-0.038765,0.081716,nan,0.000662,-0.036409,nan,nan,0.054920,0.054447,nan,0.064008,nan,0.152351,0.802468,-0.067248,-0.086388,0.281628,0.530863,0.166852,-0.417517,0.418190,-0.093303,nan,nan,nan,nan,nan,-0.194004,0.801649,-0.028213,-0.802468,0.074129,0.014574,0.833750,nan,0.006615,-0.017980,0.056622,nan,nan,-0.074287,0.218779,-0.052450,-0.007039,-0.000510,0.017392,-0.030872,0.058439,0.799192,0.007318
architecturalstyletypeid,-0.004165,-0.072694,1.000000,nan,-0.004513,0.038578,nan,nan,-0.004513,nan,nan,-0.009715,-0.009715,nan,nan,nan,nan,-0.045911,nan,-0.001420,0.023248,-0.002850,0.038688,0.078590,-0.034286,0.124977,nan,nan,nan,nan,nan,0.045670,-0.045908,-0.038229,0.045911,-0.147320,-0.003673,0.006695,nan,nan,-0.007206,nan,nan,nan,-0.044788,-0.063620,-0.061373,0.018520,0.027416,0.041283,0.006956,nan,-0.045916,-0.005919
basementsqft,0.332278,nan,nan,1.000000,0.499242,0.475352,nan,nan,0.502831,nan,0.197699,0.292052,0.292052,nan,nan,0.111523,nan,nan,0.258716,0.498234,0.321706,0.271776,nan,-0.319161,0.248058,0.054832,nan,-0.173693,nan,nan,nan,-0.048705,0.290213,0.232195,nan,0.268413,-0.370054,0.504771,nan,nan,nan,nan,0.073589,-0.042162,0.468609,0.207374,0.359519,0.431916,-0.032934,0.376552,0.439013,nan,0.289809,0.324147
bathroomcnt,-0.005498,-0.038765,-0.004513,0.499242,1.000000,0.640942,nan,0.087117,1.000000,nan,0.486178,0.772660,0.801425,-0.095007,0.585516,0.479064,-0.201148,0.011668,0.457536,0.984235,0.498065,0.450761,-0.320504,0.024473,-0.020809,0.000710,nan,0.251911,nan,nan,nan,-0.049874,0.011893,-0.037068,-0.022995,-0.031277,0.014945,0.040352,nan,0.227596,0.065576,0.085195,0.268490,0.166892,0.370355,0.325643,0.577268,0.467314,0.007514,0.348351,0.463840,0.063949,0.001963,0.029093
bedroomcnt,0.005834,0.081716,0.038578,0.475352,0.640942,1.000000,nan,0.032840,0.617967,nan,0.460816,0.631576,0.659325,-0.068031,0.566867,0.441876,-0.326976,0.028985,0.231924,0.613064,0.435494,0.597485,-0.175878,0.067736,-0.006272,-0.112666,nan,0.185181,nan,nan,nan,-0.271448,0.029644,-0.022135,-0.007211,-0.024040,0.015023,0.165843,nan,0.099854,0.041130,0.133196,0.273000,0.092578,0.044422,0.155883,0.301486,0.239563,0.008080,0.174887,0.242470,0.031352,0.008177,0.028005
buildingclasstypeid,-0.115675,nan,nan,nan,nan,nan,1.000000,-0.023729,nan,nan,nan,-0.869397,nan,nan,-0.869397,nan,nan,nan,nan,nan,nan,nan,nan,-0.137718,-0.140476,-0.972077,nan,nan,nan,nan,nan,0.034483,-0.052132,0.047645,nan,nan,-0.025387,nan,nan,nan,nan,0.061003,nan,nan,0.151196,0.061898,-0.583625,-0.316939,-0.185695,-0.077475,-0.417822,nan,-0.034122,0.122809
buildingqualitytypeid,-0.005154,0.000662,nan,na

#### If the correlation is 'nan' i.e. there is no correlation between the corresponding columns, or there is not enough data to compute the correlation, so drop those columns

In [29]:
all.drop(columns=['decktypeid','poolcnt','pooltypeid10','pooltypeid2','pooltypeid7','storytypeid'],inplace = True)

#### Dropping the below columns which the values are same in another column; calculatedfinishedsquarefeet = finishedsquarefeet15 and fullbathcnt = calculatedbathnbr

In [30]:
all.drop(columns=['finishedsquarefeet15'],inplace = True)
all.drop(columns=['calculatedbathnbr'],inplace = True)

In [31]:
all.columns

Index(['parcelid', 'airconditioningtypeid', 'architecturalstyletypeid',
       'basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingclasstypeid',
       'buildingqualitytypeid', 'finishedfloor1squarefeet',
       'calculatedfinishedsquarefeet', 'finishedsquarefeet12',
       'finishedsquarefeet13', 'finishedsquarefeet50', 'finishedsquarefeet6',
       'fips', 'fireplacecnt', 'fullbathcnt', 'garagecarcnt',
       'garagetotalsqft', 'hashottuborspa', 'heatingorsystemtypeid',
       'latitude', 'longitude', 'lotsizesquarefeet', 'poolsizesum',
       'propertycountylandusecode', 'propertylandusetypeid',
       'propertyzoningdesc', 'rawcensustractandblock', 'regionidcity',
       'regionidcounty', 'regionidneighborhood', 'regionidzip', 'roomcnt',
       'threequarterbathnbr', 'typeconstructiontypeid', 'unitcnt',
       'yardbuildingsqft17', 'yardbuildingsqft26', 'yearbuilt',
       'numberofstories', 'fireplaceflag', 'structuretaxvaluedollarcnt',
       'taxvaluedollarcnt', 'assessmentyea

### Checking for missing values and imputing missing values using different techniques like KNN, mean, median and most_frequent

In [32]:
print(all['calculatedfinishedsquarefeet'].isnull().sum())
print(all['fullbathcnt'].isnull().sum())
print(all['regionidzip'].isnull().sum())
print(all['structuretaxvaluedollarcnt'].isnull().sum())
print(all['taxvaluedollarcnt'].isnull().sum())
print(all['landtaxvaluedollarcnt'].isnull().sum())
print(all['taxamount'].isnull().sum())
print(all['yearbuilt'].isnull().sum())
print(all['censustractandblock'].isnull().sum())
print(all['regionidcity'].isnull().sum())
print(all['buildingqualitytypeid'].isnull().sum())
print(all['heatingorsystemtypeid'].isnull().sum())
print(all['lotsizesquarefeet'].isnull().sum())
print(all['propertyzoningdesc'].isnull().sum())
print(all['regionidneighborhood'].isnull().sum())
print(all['unitcnt'].isnull().sum())
print(all['propertycountylandusecode'].isnull().sum())

868
1788
116
518
35
36
44
1028
869
3269
59844
61310
18155
58264
99382
57995
34


In [33]:
print(all['airconditioningtypeid'].isnull().sum())
print(all['architecturalstyletypeid'].isnull().sum())
print(all['basementsqft'].isnull().sum())
print(all['buildingclasstypeid'].isnull().sum())
print(all['finishedfloor1squarefeet'].isnull().sum())
print(all['finishedsquarefeet12'].isnull().sum())
print(all['finishedsquarefeet13'].isnull().sum())
print(all['finishedsquarefeet50'].isnull().sum())
print(all['finishedsquarefeet6'].isnull().sum())
print(all['fireplacecnt'].isnull().sum())
print(all['garagecarcnt'].isnull().sum())
print(all['garagetotalsqft'].isnull().sum())
print(all['hashottuborspa'].isnull().sum())
print(all['poolsizesum'].isnull().sum())
print(all['threequarterbathnbr'].isnull().sum())
print(all['typeconstructiontypeid'].isnull().sum())
print(all['yardbuildingsqft17'].isnull().sum())
print(all['yardbuildingsqft26'].isnull().sum())
print(all['numberofstories'].isnull().sum())
print(all['fireplaceflag'].isnull().sum())
print(all['taxdelinquencyflag'].isnull().sum())
print(all['taxdelinquencyyear'].isnull().sum())

112153
164751
165118
165180
152476
8192
165137
152476
164416
147533
110515
110515
161352
163398
143339
164698
160229
165048
127600
164825
160681
160681


In [34]:
# create a KNNImputer object
knnimputer = KNNImputer(n_neighbors=2)

# impute the missing values in a particular column
all['regionidzip'] = knnimputer.fit_transform(all[['regionidzip']])
all['regionidcounty'] = knnimputer.fit_transform(all[['regionidcounty']])
all['latitude'] = knnimputer.fit_transform(all[['latitude']])
all['longitude'] = knnimputer.fit_transform(all[['longitude']])

In [35]:
# Create a SimpleImputer object with strategy='mean'
imputer = SimpleImputer(strategy='mean')

# Impute missing values in a particular column
all['taxvaluedollarcnt'] = imputer.fit_transform(all[['taxvaluedollarcnt']])
all['landtaxvaluedollarcnt'] = imputer.fit_transform(all[['landtaxvaluedollarcnt']])
all['taxamount'] = imputer.fit_transform(all[['taxamount']])
all['structuretaxvaluedollarcnt'] = imputer.fit_transform(all[['structuretaxvaluedollarcnt']])
all['lotsizesquarefeet'] = imputer.fit_transform(all[['lotsizesquarefeet']])
all['rawcensustractandblock'] = imputer.fit_transform(all[['rawcensustractandblock']])

In [36]:
# Create a SimpleImputer object with strategy='mean'
imputer = SimpleImputer(strategy='most_frequent')

# Impute missing values in a particular column
all['yearbuilt'] = imputer.fit_transform(all[['yearbuilt']])
all['censustractandblock'] = imputer.fit_transform(all[['censustractandblock']])
all['propertycountylandusecode'] = imputer.fit_transform(all[['propertycountylandusecode']])
all['regionidneighborhood'] = imputer.fit_transform(all[['regionidneighborhood']])
all['regionidcity'] = knnimputer.fit_transform(all[['regionidcity']])
all['fullbathcnt'] = imputer.fit_transform(all[['fullbathcnt']])
all['buildingqualitytypeid'] = imputer.fit_transform(all[['buildingqualitytypeid']])
all['heatingorsystemtypeid'] = imputer.fit_transform(all[['heatingorsystemtypeid']])
all['propertyzoningdesc'] = imputer.fit_transform(all[['propertyzoningdesc']])
all['unitcnt'] = imputer.fit_transform(all[['unitcnt']])
all['airconditioningtypeid'] = imputer.fit_transform(all[['airconditioningtypeid']])
all['garagecarcnt'] = imputer.fit_transform(all[['garagecarcnt']])
all['assessmentyear'] = imputer.fit_transform(all[['assessmentyear']])
all['roomcnt'] = imputer.fit_transform(all[['roomcnt']])
all['bathroomcnt'] = imputer.fit_transform(all[['bathroomcnt']])
all['bedroomcnt'] = imputer.fit_transform(all[['bedroomcnt']])
all['fips'] = imputer.fit_transform(all[['fips']])
all['propertylandusetypeid'] = imputer.fit_transform(all[['propertylandusetypeid']])

In [37]:
# Create a SimpleImputer object with strategy='mean'
imputer = SimpleImputer(strategy='median')

# Impute missing values in a particular column
all['calculatedfinishedsquarefeet'] = imputer.fit_transform(all[['calculatedfinishedsquarefeet']])
all['finishedsquarefeet12'] = imputer.fit_transform(all[['finishedsquarefeet12']])
all['garagetotalsqft'] = imputer.fit_transform(all[['finishedsquarefeet12']])

#### Dropping columns which has a threshold of 75% missing data

In [38]:
def drop_columns(data_series):
    missing_value_df = pd.DataFrame((data_series.isnull().sum()/len(data_series))*100,columns=["missing_value"])
    drop_columns_list = missing_value_df.loc[missing_value_df["missing_value"]>75].index.to_list()
    return drop_columns_list

In [39]:
drp_col = drop_columns(all)
len(drp_col)
# drp_col

18

In [40]:
all.drop(columns=drp_col,inplace = True)

#### Converting objects to categorical codes for regression

In [41]:
for column in ('propertycountylandusecode', 'propertyzoningdesc'):
    all[column] = pd.Categorical(all[column]).codes

In [42]:
corr = all.corr()
corr.style.background_gradient(cmap='coolwarm')

,parcelid,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,garagecarcnt,garagetotalsqft,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,unitcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror
parcelid,1.000000,0.062408,-0.005478,0.005836,0.143151,0.020331,0.032266,0.545724,-0.042365,-0.108475,0.032266,-0.040573,-0.196281,-0.016274,-0.057375,0.461526,0.037204,-0.127195,0.545231,-0.013400,-0.423121,-0.064451,0.035492,0.399099,-0.019797,0.144564,0.020056,0.008841,0.002876,0.002775,-0.012058,0.104346,0.012268
airconditioningtypeid,0.062408,1.000000,0.029956,0.049399,0.059777,0.060669,0.066016,0.073192,0.007528,0.026751,0.066016,-0.042734,-0.142452,0.162712,-0.027787,0.201460,-0.027354,-0.065478,0.071835,-0.020713,-0.232455,-0.030992,0.013982,0.305120,-0.010921,0.037373,-0.004360,0.006154,0.000303,0.010275,-0.007710,0.013692,0.003976
bathroomcnt,-0.005478,0.029956,1.000000,0.640939,0.072929,0.765215,0.736546,0.011701,0.962403,0.224654,0.736546,-0.257512,0.024473,-0.020809,0.000684,0.043666,-0.049864,0.007308,0.011893,-0.036499,-0.022995,-0.021679,0.014936,0.040381,0.070354,0.371192,0.573754,0.467310,0.007568,0.348343,0.463830,0.002364,0.029114
bedroomcnt,0.005836,0.049399,0.640939,1.000000,0.023623,0.623635,0.560910,0.028989,0.597101,0.198107,0.560910,-0.150049,0.067736,-0.006272,-0.105809,0.017062,-0.271446,0.023043,0.029644,-0.021870,-0.007211,-0.060245,0.015014,0.165843,0.109660,0.050972,0.299204,0.239562,0.008086,0.174882,0.242462,0.008785,0.027994
buildingqualitytypeid,0.143151,0.059777,0.072929,0.023623,1.000000,0.094362,0.103420,0.227582,0.055456,-0.068564,0.103420,-0.053744,-0.150934,0.058145,-0.023011,0.278013,0.026559,-0.108596,0.226565,-0.027886,-0.282872,-0.033294,0.021296,0.208256,-0.013583,0.103230,0.118314,0.118737,0.195153,0.102671,0.101034,0.043073,-0.004509
calculatedfinishedsquarefeet,0.020331,0.060669,0.765215,0.623635,0.094362,1.000000,0.949496,0.058029,0.772940,0.259192,0.949496,-0.213567,0.006767,-0.020394,-0.049168,0.058782,-0.211017,-0.020493,0.058073,-0.040269,-0.063728,-0.018677,0.023161,0.097882,0.091225,0.214325,0.698213,0.585868,0.005463,0.449288,0.582708,0.010391,0.039895
finishedsquarefeet12,0.032266,0.066016,0.736546,0.560910,0.103420,0.949496,1.000000,0.079437,0.729173,0.282387,1.000000,-0.211746,0.008875,-0.018612,-0.044651,0.038491,-0.080849,-0.016507,0.079550,-0.039711,-0.087007,-0.039785,0.025497,0.123298,-0.019077,0.259492,0.715651,0.599410,0.006965,0.459028,0.593306,0.014599,0.041914
fips,0.545724,0.073192,0.011701,0.028989,0.227582,0.058029,0.079437,1.000000,-0.043783,-0.148666,0.079437,-0.119898,-0.072683,-0.327255,-0.074849,0.704034,0.054318,-0.248691,0.999904,-0.032146,-0.608673,-0.120553,0.052013,0.703750,-0.035328,0.235138,0.030975,0.009151,-0.002356,-0.002592,-0.024195,0.191227,0.009604
fullbathcnt,-0.042365,0.007528,0.962403,0.597101,0.055456,0.772940,0.729173,-0.043783,1.000000,0.212467,0.729173,-0.263016,0.076468,-0.056404,0.011329,-0.025180,-0.100600,0.032534,-0.043222,-0.035410,0.057797,-0.014959,0.010208,-0.022102,0.101900,0.324792,0.576791,0.468989,0.003424,0.350839,0.470995,-0.008971,0.028694
garagecarcnt,-0.108475,0.026751,0.224654,0.198107,-0.068564,0.259192,0.282387,-0.148666,0.212467,1.000000,0.282387,-0.054057,0.145810,-0.081517,0.026806,-0.268206,-0.061158,0.074924,-0.147656,0.010098,0.242017,0.020188,-0.016601,-0.056880,-0.015171,0.006436,0.218886,0.154350,0.001744,0.101367,0.152201,-0.028359,0.012434


In [43]:
all.drop(columns=["transactiondate"],inplace=True)
all.set_index('parcelid',inplace= True)
all.head()

,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,garagecarcnt,garagetotalsqft,...,roomcnt,unitcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror
parcelid,,,,,,,,,,,,,,,,,,,,,
17073783,1.0,2.5,3.0,7.0,1264.0,1264.0,6111.0,2.0,2.0,1264.0,...,5.0,1.0,1986.0,115087.0,191811.0,2015.0,76724.0,2015.06,6.111002e+13,0.0953
14762478,1.0,3.0,4.0,7.0,2655.0,2655.0,6059.0,3.0,2.0,2655.0,...,0.0,1.0,1985.0,241544.0,397303.0,2015.0,155759.0,4099.00,6.059063e+13,0.8519
14743592,1.0,2.0,2.0,7.0,847.0,847.0,6059.0,2.0,1.0,847.0,...,0.0,1.0,1993.0,115279.0,211050.0,2015.0,95771.0,2863.96,6.059053e+13,-0.1393
14729860,1.0,2.0,2.0,7.0,862.0,862.0,6059.0,2.0,1.0,862.0,...,0.0,1.0,1989.0,94935.0,298480.0,2015.0,203545.0,3081.24,6.059042e+13,-0.0212
14721734,1.0,2.0,2.0,7.0,938.0,938.0,6059.0,2.0,1.0,938.0,...,0.0,1.0,1990.0,111769.0,354427.0,2015.0,242658.0,4094.74,6.059076e+13,0.0459


## SPLIT train and test (49/51% split)

In [44]:
y_all = all[['logerror']]
x_all = all.drop('logerror', axis=1).copy()
x_all

,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,garagecarcnt,garagetotalsqft,...,regionidzip,roomcnt,unitcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock
parcelid,,,,,,,,,,,,,,,,,,,,,
17073783,1.0,2.5,3.0,7.0,1264.0,1264.0,6111.0,2.0,2.0,1264.0,...,97081.000000,5.0,1.0,1986.0,115087.000000,191811.000000,2015.0,76724.000000,2015.060000,6.111002e+13
14762478,1.0,3.0,4.0,7.0,2655.0,2655.0,6059.0,3.0,2.0,2655.0,...,96951.000000,0.0,1.0,1985.0,241544.000000,397303.000000,2015.0,155759.000000,4099.000000,6.059063e+13
14743592,1.0,2.0,2.0,7.0,847.0,847.0,6059.0,2.0,1.0,847.0,...,96943.000000,0.0,1.0,1993.0,115279.000000,211050.000000,2015.0,95771.000000,2863.960000,6.059053e+13
14729860,1.0,2.0,2.0,7.0,862.0,862.0,6059.0,2.0,1.0,862.0,...,96987.000000,0.0,1.0,1989.0,94935.000000,298480.000000,2015.0,203545.000000,3081.240000,6.059042e+13
14721734,1.0,2.0,2.0,7.0,938.0,938.0,6059.0,2.0,1.0,938.0,...,97018.000000,0.0,1.0,1990.0,111769.000000,354427.000000,2015.0,242658.000000,4094.740000,6.059076e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11928618,1.0,2.0,3.0,7.0,1543.0,1522.0,6037.0,2.0,2.0,1522.0,...,96587.665639,0.0,1.0,1955.0,184847.890596,473840.705458,2015.0,289535.094664,5999.531543,6.037137e+13
10813418,1.0,2.0,3.0,7.0,1543.0,1522.0,6037.0,2.0,2.0,1522.0,...,96587.665639,0.0,1.0,1955.0,184847.890596,473840.705458,2015.0,289535.094664,5999.531543,6.037137e+13
11934023,1.0,2.0,3.0,7.0,1543.0,1522.0,6037.0,2.0,2.0,1522.0,...,96587.665639,0.0,1.0,1955.0,184847.890596,473840.705458,2015.0,289535.094664,5999.531543,6.037137e+13


In [45]:
X_train, X_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.51)

In [46]:
print(len(x_all),len(X_train),len(X_test),len(y_train),len(y_test))

165210 80952 84258 80952 84258


In [47]:
x_all.dtypes

airconditioningtypeid           float64
bathroomcnt                     float64
bedroomcnt                      float64
buildingqualitytypeid           float64
calculatedfinishedsquarefeet    float64
finishedsquarefeet12            float64
fips                            float64
fullbathcnt                     float64
garagecarcnt                    float64
garagetotalsqft                 float64
heatingorsystemtypeid           float64
latitude                        float64
longitude                       float64
lotsizesquarefeet               float64
propertycountylandusecode          int8
propertylandusetypeid           float64
propertyzoningdesc                int16
rawcensustractandblock          float64
regionidcity                    float64
regionidcounty                  float64
regionidneighborhood            float64
regionidzip                     float64
roomcnt                         float64
unitcnt                         float64
yearbuilt                       float64


# Fitting a regression model and predict the log loss. Evaluate each model's performance using mean absolute error.

#### Decision Tree Regression

In [48]:
regressor = DecisionTreeRegressor(max_depth=6,random_state=0).fit(X_train, y_train)
y_pred = regressor.predict(X_test)
print("DecisionTreeRegressor MAE",metrics.mean_absolute_error(y_test,y_pred))

DecisionTreeRegressor MAE 0.06946074376665157


#### Linear Regression

In [49]:
reg = LinearRegression().fit(X_train, y_train)
y_pred = reg.predict(X_test)
print("LinearRegression MAE",metrics.mean_absolute_error(y_test,y_pred))

LinearRegression MAE 0.06900497168589419


#### Gradient Boosting Regression

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor()

# fit the model to the training data
gbr.fit(X_train, y_train)

# make predictions on the test data
y_pred = gbr.predict(X_test)

# calculate the mean squared error on the test data
print("GradientBoostingRegressor MAE",metrics.mean_absolute_error(y_test,y_pred))

#### Random Forest Regression

In [ ]:
regr = RandomForestRegressor(max_depth=5, random_state=0)
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)
print("RandomForestRegressor MAE",metrics.mean_absolute_error(y_test,y_pred))

### Writing the best model to Pickle file

In [ ]:
filename = 'zillow.model'
pickle.dump(regr, open(filename, 'wb'))